In [1]:
pip install ollama

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from ollama import Client

In [3]:
f = open('data.json', 'r')
data = json.load(f)
f.close()

Extrahieren der IDs und Dialogtexte aus dem MultiWOZ-Datensatz

In [4]:
dialogs = {}

for key, value in data.items():
    dialogs[key] = []
    for entry in value["log"]:
        dialogs[key].append(entry["text"])

In [5]:
count = 0
for key, dialog in dialogs.items():
    print(f"{key}: {dialog}\n")
    count += 1
    if count == 10:
        break

SNG01856.json: ['am looking for a place to to stay that has cheap price range it should be in a type of hotel', 'Okay , do you have a specific area you want to stay in ?', "no , i just need to make sure it 's cheap . oh , and i need parking", 'I found 1 cheap hotel for you that includes parking . Do you like me to book it ?', 'Yes , please . 6 people 3 nights starting on tuesday .', "I am sorry but I was n't able to book that for you for Tuesday . Is there another day you would like to stay or perhaps a shorter stay ?", 'how about only 2 nights .', 'Booking was successful . \n Reference number is : 7GAWK763 . Anything else I can do for you ?', 'No , that will be all . Good bye .', 'Thank you for using our services .']

SNG0129.json: ['Hello , I have been robbed .   Can you please help me get in touch with the police ?', 'Parkside Police Station is in Parkside , Cambridge . Their number is 01223358966 . Anything else I can do for you ?', 'Can I please have the postcode as well ?', 'The 

In [6]:
llm_prompt = """
Here follows a list of dialogues between a user and an assistant-bot. The first utterance is always from the user, followed by the bot, and so on.
Please analyze if this conversation is coherent and rate it on a coherence-score from 1 to 5, where:
1 = Absolutely incoherent
5 = Perfectly coherent

Provide an explanation for your score. Follow these rules when determining coherence:
1. A dialogue is coherent if the bot completes the task set by the user.
2. A dialogue is coherent if it is natural, meaning the bot does not repeat itself excessively, avoids robotic behavior, and maintains logical consistency.
3. The dialogue should be logically connected, with responses building on previous turns. This should be given the most weight in scoring.
4. The conversation should logically adhere to achieving the user’s goal step by step.

Additionally, identify the assistance functions the bot performs in the dialogue. A bot can perform multiple functions.
Possible assistance functions: 
1. Activation assistance: Creation of an optimal activation level.
2. Coach assistance: Reinforcement of a motif.
3. Warning assistance: Inhibition of a motif.
4. Orientation assistance: Suggestion of a change of destination.
5. Explanation assistance: Explanation of system outputs.
6. Offer assistance: Information about all options.
7. Filter assistance: Information about selected options.
8. Consultant assistance: Suggestion of an option.
9. Delegation assistance: Proposal and realisation, if the user agrees.
10. Takeover assistance: Proposal and realisation, if the user does not disagree.
11. Informative execution assistance: Realisation with information for the user.
12. Silent execution assistance: Realisation without information for the user.
13. Power assistance: Reinforcing actions.
14. Short cut assistance: Shortening an action sequence.
15. Input assistance: Provide alternative modalities.
16. Feedback assistance: Making effects perceptible.
17. Critique assistance: Evaluate the degree of target achievement.

Output must be in this exact JSON format: (Any deviation from this format is an error.)
{"id": "<EXACT ID FROM INPUT, e.g., 'SSNG0362.json'>",  
"text": ["Hello , I have been robbed . Can you please help me get in touch with the police ?",  
         "Parkside Police Station is in Parkside , Cambridge . Their number is 01223358966 . Anything else I can do for you ?",  
         "Can I please have the postcode as well ?",  
         "The postcode for the Parkside Police Station is CB11JG . Can I help you with anything else ?",  
         "Was Parkside the address of the police station ? If not , can I have the address please ?",  
         "Yes , Parkside is the address .",  
         "Thank you that will be all for now .",  
         "Great . Thank you for contacting Cambridge Towninfo Centre .",  
         "You were great . Goodbye .",  
         "We are happy to help . Have a good day !"],
"score": 5,  
"explanation of score": "The dialogue is coherent and natural. The bot provides relevant information to the user's request, answering their questions and providing the necessary details. The conversation flows logically, with each response building upon the previous one. Overall, the dialogue is well-structured and effectively addresses the user's concerns.",  
"assistance functions": ["Orientation assistance", "Explanation assistance", "Filter assistance"]  
}

Rules for output format:
The "id" must match the dialogue name in the input dictionary. Do not reuse any example "id" from this prompt.
Each "id" is unique. If "id" does not match the input, it is an error.
The response must be a single, valid JSON object. No extra text outside of JSON.
Each key must have the correct type:
"id" → A string in "SSNGXXXX.json" format,taken from input.
"text" → A list of strings, preserving the original dialogue.
"score" → An integer (1, 2, 3, 4, or 5—no other values).
"explanation of score" → A well-reasoned explanation.
"assistance functions" → A list containing only predefined assistance functions.
Strict JSON formatting: The response should be machine-readable JSON without missing brackets, extra text, or invalid syntax.
Step-by-step generation: First extract the dialogue, then determine the score, then explain, then identify functions, then format in JSON.
Before outputting, check that the JSON is valid.
"""

Übergabe der Daten und des Prompts an das Modell
Speichern der Ergebnisse in einer Liste

In [ ]:
llm = Client(host="http://hcai.uni-regensburg.de:11434")
liste = []
for key, value in dialogs.items():
    dic_test = {}
    dic_test[key] = value
    mess = llm.chat(model='llama3.3:latest', #kleines Modell: llama3:latest
                messages=[{'role': 'system',
                            'content': llm_prompt + str(dic_test)},
                         ])
    liste.append(mess['message']['content'])

In [ ]:
print(len(liste))

In [ ]:
print(liste)

In [20]:
with open('run_number_llama3-70b.txt', 'w+') as f:
    for items in liste:
        f.write('%s\n' %items)
    
    print("File written successfully")

f.close() 

File written successfully
